# <font style="color:rgb(50, 120, 229);">  Regularización en Keras </font>



En este cuaderno, adoptaremos un enfoque experimental para aprender cómo usar conceptos para mejorar nuestro modelo. 

Ilustraremos las diferentes técnicas usando el conjunto de datos Fashion MNIST.
<center>
<img src="./images/MNIST_fashion.png" width="500">
</center>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Flatten, Input, Normalization, Resizing
from matplotlib import pyplot as plt

plt.style.use('ggplot')

In [ ]:
def plot_metric(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    plt.figure(figsize=(10, 5))
    plt.suptitle(title)

    plt.subplot(2, 1, 1)
    plt.plot(loss, label='loss')
    plt.plot(val_loss, label='val_loss')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(acc, label='accuracy')
    plt.plot(val_acc, label='val_accuracy')
    plt.legend()


In [ ]:
from keras.datasets import fashion_mnist
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## <font style="color:rgb(50, 120, 229);">  Experimento 1: Usar un modelo simple </font>

Comienza con una CNN pequeña (SmallModel [S]). 

Ten en cuenta que si la precisión de entrenamiento no es muy alta, el modelo no es lo suficientemente complejo como para sobreajustar (obtener casi un 100% de precisión en los datos de entrenamiento o alrededor de cero pérdida en los datos de entrenamiento). 

Siguiendo la definición de sesgo y varianza, este modelo tiene sesgo alto y varianza alta.

In [ ]:
simple_model = Sequential([
   Input(shape=(28, 28, 1)),
   Normalization(),
   Conv2D(filters=4, kernel_size=5, activation='relu'),
   MaxPooling2D(2),
   Conv2D(filters=8, kernel_size=3, activation='relu'),
   MaxPooling2D(2),
   Flatten(),
   Dense(100, activation='relu'),
   Dense(50, activation='relu'),
   Dense(10, activation='softmax')
])

simple_model.summary()

In [ ]:
simple_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
simple_history = simple_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)

In [ ]:
plot_metric(simple_history, 'Simple model')

## <font style="color:rgb(50, 120, 229);">  Experimento 2: Aumentar la complejidad del modelo </font>

Primero, entrenaremos un modelo sin regularización (sin Dropout, Batch Norm, Aumento de datos, Programador de velocidad de aprendizaje o penalización L2). 

Descubrimos que ese modelo sobreajusta (100% de precisión en el entrenamiento, pero solo tiene 92-93% para la validación). Este modelo tiene sesgo bajo y varianza alta. Por lo tanto, necesitamos usar técnicas de regularización para reducir esta alta varianza.


In [ ]:
mid_model = Sequential([
    Input(shape=(28, 28, 1)),
    Normalization(),
    Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

mid_model.summary()

In [ ]:
mid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mid_history = mid_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)

In [ ]:
plot_metric(mid_history, 'Mid model')

## <font style="color:rgb(50, 120, 229);">  Experimento 3: Agregar Batch Normalization </font>

En este experimento, agregaremos capas `BatchNormalization` a la parte del extractor de características del modelo. 

Notarás que la precisión de validación mejora, así como la pérdida disminuye. **Incluso si la precisión no mejora y solo la pérdida disminuye, aún significa que este es un modelo más robusto** (la confianza en la clasificación incorrecta no será muy alta).


In [ ]:
mid_model_batch_norm = Sequential([
    Input(shape=(28, 28, 1)),
    Normalization(),
    Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

mid_model_batch_norm.summary()

In [ ]:
mid_model_batch_norm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mid_history_batch_norm = mid_model_batch_norm.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)

In [ ]:
plot_metric(mid_history_batch_norm, 'Mid model with batch normalization')

## <font style="color:rgb(50, 120, 229);">  Experimento 4: Agregar Dropout </font>

En lugar de capas de normalización por lotes, utilizaremos Dropout2d para capas convolucionales y Dropout para capas lineales. 

Con las capas de dropout, la precisión de validación permanece casi igual, pero la pérdida mejora.

In [ ]:
mid_model_dropout = Sequential([
    Input(shape=(28, 28, 1)),
    Normalization(),
    Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Dropout(0.5),
    Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Dropout(0.5),
    Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Dropout(0.5),
    Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

mid_model_dropout.summary()

In [ ]:
mid_model_dropout.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mid_history_dropout = mid_model_dropout.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)

In [ ]:
plot_metric(mid_history_dropout, 'Mid model with dropout')

## <font style="color:rgb(50, 120, 229);">  Experimento 5: Batch Normalization y Dropout </font>

Ahora, considera Batch Norm con Dropout. En términos de precisión de validación y pérdida, estos casos son ligeramente mejores que los anteriores.

In [ ]:
mid_model_dropout_batch_norm = Sequential([
    Input(shape=(28, 28, 1)),
    Normalization(),
    Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.5),
    Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.5),
    Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.5),
    Conv2D(filters=512, kernel_size=3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

mid_model_dropout_batch_norm.summary()

In [ ]:
mid_model_dropout_batch_norm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mid_history_dropout_batch_norm = mid_model_dropout_batch_norm.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)

In [ ]:
plot_metric(mid_history_dropout_batch_norm, 'Mid model with dropout and batch normalization')